<a href="https://colab.research.google.com/github/1004516/SE-ALES-Y-SISTEMAS/blob/main/Simulacion%20punto%204%20parcial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import sympy as sp

class FourierSeriesCalculator:
    # Calcula y verifica los coeficientes de la Serie de Fourier para una señal
    # triangular asimétrica y la reconstruye.

    def __init__(self, A, T, d1, d2, N):
        # Inicializa la calculadora con los parámetros de la señal.

        if not (0 < d1 < d2 < T / 2):
            raise ValueError("Los parámetros deben cumplir 0 < d1 < d2 < T/2")
        self.A, self.T, self.d1, self.d2, self.N = A, T, d1, d2, N
        self.w0 = 2 * np.pi / self.T
        self.n_values = np.arange(-self.N, self.N + 1)

    def calculate_coefficients(self):
        # Calcula el espectro de coeficientes Cn usando el método de la derivada.

        C0 = (self.A * self.d2) / self.T
        n_nonzero = self.n_values[self.n_values != 0]
        n_w0 = n_nonzero * self.w0
        term_dn = (
            (1 / self.d1)
            - (self.d2 / (self.d1 * (self.d2 - self.d1))) * np.cos(n_w0 * self.d1)
            + (1 / (self.d2 - self.d1)) * np.cos(n_w0 * self.d2)
        )
        Dn = (2 * self.A / self.T) * term_dn
        Cn_nonzero = -Dn / (n_w0**2)
        Cn_spectrum = np.zeros(2 * self.N + 1, dtype=float)
        zero_index = self.N
        Cn_spectrum[zero_index] = C0
        Cn_spectrum[:zero_index] = Cn_nonzero[:zero_index]
        Cn_spectrum[zero_index + 1:] = Cn_nonzero[zero_index:]
        print(f"Cálculo optimizado de {2*self.N+1} coeficientes Cn realizado.")
        return Cn_spectrum

    def verify_with_sympy(self):
        # Verifica los coeficientes Cn para n!=0 usando la integral directa con SymPy.

        # Setup de variables simbólicas (locales a este método)
        t, n = sp.symbols('t n', real=True)
        A_s, T_s, d1_s, d2_s, w0_s = sp.symbols('A T d1 d2 w0', real=True)

        # Definición simbólica de la señal x(t) para t en [0, T/2]
        m_in = A_s / d1_s
        m_out = A_s / (d2_s - d1_s)
        x_t1 = m_in * t
        x_t2 = A_s - m_out * (t - d1_s)

        # Cálculo de la integral simbólica para un 'n' genérico
        # C_n = (2/T) * Int[0, T/2] x(t) * cos(n*w0*t) dt
        integral1 = sp.integrate(x_t1 * sp.cos(n * w0_s * t), (t, 0, d1_s))
        integral2 = sp.integrate(x_t2 * sp.cos(n * w0_s * t), (t, d1_s, d2_s))

        # Fórmula simbólica final para Cn (n!=0)
        Cn_formula = (2 / T_s) * (integral1 + integral2)

        # Sustitución y Evaluación Numérica
        params = {A_s: self.A, T_s: self.T, d1_s: self.d1, d2_s: self.d2, w0_s: self.w0}
        Cn_formula_subs = Cn_formula.subs(params)

        # Convertir la fórmula simbólica en una función numérica rápida
        # Esto es mucho más eficiente que sustituir n en un bucle
        n_nonzero = self.n_values[self.n_values != 0]
        Cn_numeric_func = sp.lambdify(n, Cn_formula_subs, 'numpy')
        Cn_sympy_nonzero = Cn_numeric_func(n_nonzero)

        # Ensamblaje del Espectro Completo
        Cn_sympy = np.zeros(2 * self.N + 1, dtype=float)
        zero_index = self.N
        Cn_sympy[zero_index] = (self.A * self.d2) / self.T # C0
        Cn_sympy[:zero_index] = Cn_sympy_nonzero[:zero_index]
        Cn_sympy[zero_index + 1:] = Cn_sympy_nonzero[zero_index:]

        print("Cálculo de comprobación con SymPy realizado.")
        return Cn_sympy

    @staticmethod
    def reconstruct_signal(t_array, C_n_array, n_values, w0_val):
        # Reconstruye la señal usando la serie de Fourier de forma vectorizada.

        # Fórmula: x(t) = Σ[n=-N, N] C_n * exp(j*n*w0*t)

        # Se usan np.newaxis para alinear los arrays n y t para una
        # operación vectorizada (broadcasting)

        n_col = n_values[:, np.newaxis]
        Cn_col = C_n_array[:, np.newaxis]

        # Matriz de exponenciales complejas: una fila por cada 'n', una columna por cada 't'
        exponentials = np.exp(1j * n_col * w0_val * t_array)

        # Suma ponderada de las exponenciales a lo largo del eje 'n' (axis=0)
        x_rec_complex = np.sum(Cn_col * exponentials, axis=0)

        return np.real(x_rec_complex)


# 2. EJECUCIÓN DEL CÁLCULO Y LA COMPROBACIÓN
# ==========================================

if __name__ == "__main__":
    # Parámetros de la Señal
    A_val, T_val, d1_val, d2_val, N_val = 5.0, 10.0, 1.0, 4.0, 5

    # 1. Crear una instancia de la calculadora
    signal_analyzer = FourierSeriesCalculator(A=A_val, T=T_val, d1=d1_val, d2=d2_val, N=N_val)

    # 2. Calcular los coeficientes con el método principal
    Cn_spectrum = signal_analyzer.calculate_coefficients()

    # 3. Verificar los coeficientes usando SymPy
    Cn_sympy = signal_analyzer.verify_with_sympy()

    # 4. Comparar los resultados
    max_diff = np.max(np.abs(Cn_spectrum - Cn_sympy))
    print(f"\nComprobación: Máxima diferencia entre métodos = {max_diff:.2e}")

    # 5. Reconstruir la señal
    t_points = np.linspace(0, T_val, 500)
    x_reconstructed = signal_analyzer.reconstruct_signal(
      t_array=t_points,
      C_n_array=Cn_spectrum,
      n_values=signal_analyzer.n_values,
      w0_val=signal_analyzer.w0
    )


print("\n--- Cálculo de Potencia y Error")

# 1. Definir la variable simbólica de SymPy para la integración
t = sp.symbols('t', real=True, positive=True)

# 2. Definir las pendientes de la señal con valores numéricos
m_in = A_val / d1_val
m_out = A_val / (d2_val - d1_val)

# 3. Expresiones para cada tramo de la señal x(t)
x_t_in = m_in * t
x_t_out = A_val - m_out * (t - d1_val)

# 4. Calcular la integral de x(t)^2 en cada tramo
# Tramo 1: de 0 a d1
integral1 = sp.integrate(x_t_in**2, (t, 0, d1_val))
# Tramo 2: de d1 a d2
integral2 = sp.integrate(x_t_out**2, (t, d1_val, d2_val))

# 5. Calcular la potencia total P_x
# La integral sobre un periodo es 2 * (integral1 + integral2) por la simetría par.
P_x = (1 / T_val) * 2 * (integral1 + integral2)
P_x_numeric = float(P_x) # Convertir a valor numérico final

# b. Potencia de la Serie Parcial P_FS (Teorema de Parseval)
# P_FS = Sum(|Cn|^2) para n de -N a N
P_FS = np.sum(np.abs(Cn_spectrum)**2) # Use Cn_spectrum from the main calculation

# c. Error Relativo
# Comparamos la potencia calculada en el dominio del tiempo vs. dominio de la frecuencia
E_R = np.abs(P_x_numeric - P_FS) / P_x_numeric

print(f"Potencia Total (vía SymPy): {P_x_numeric:.8f}")
print(f"Potencia de la Serie Parcial (vía Parseval): {P_FS:.8f}")
print(f"Error Relativo calculado: {E_R*100:.4f}%")

Cálculo optimizado de 11 coeficientes Cn realizado.
Cálculo de comprobación con SymPy realizado.

Comprobación: Máxima diferencia entre métodos = 2.16e-15

--- Cálculo de Potencia y Error
Potencia Total (vía SymPy): 6.66666667
Potencia de la Serie Parcial (vía Parseval): 6.61391486
Error Relativo calculado: 0.7913%
